In [17]:
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import mediapipe as mp
import torch.nn.functional as F
import cv2
import time
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [3]:
data, temp = [],[]
gests = os.listdir('dataset')
for gest in gests:
    data = []
    print(gest)
    f = open('dataset/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        '''
        D = []
        x = f[i][0]
        y = f[i][1]
        rs_x = 1 - x
        rs_y = 1 - y
        for j in range(0, len(f[i]), 2):
            D.append(f[i][j] + rs_x)
            D.append(f[i][j + 1] + rs_y)
        '''
        data.append(f[i])
    temp.append(data)
data = np.array(temp)

leftkulak.txt
leftladon.txt
l_arm0.txt
rkulak.txt
rladon.txt
r_arm0.txt


<ipython-input-3-184b4d701883>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(temp)


In [4]:
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

1080
577
557
956
1375
575


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 6)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x

In [6]:
net = Net()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

In [8]:
for epoch in range(15):
    running_loss = 0.0
    for samples, labels in dataloader:
        pred = net(samples)
        loss = criterion(pred, labels)
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
    print(running_loss/len(dataloader))
print('Finished Training')

0.9885902244132012
0.3045117908390239
0.1825225792475976
0.12501876802998596
0.10246012363350018
0.07506021868175594
0.07662174387223786
0.07008443461163552
0.07978963340647169
0.06621528181349276
0.054224022781272654
0.059562870741137884
0.05029345959737839
0.053970273863524196
0.05119201993729803
Finished Training


In [9]:
ls=0
for samples, labels in dataloader:
    pred = net(samples)
    v,i = torch.max(pred, 1)
    ls+=torch.sum(abs(labels - i)).item()

print((len(dataset)-ls)/len(dataset))


0.975390625


In [11]:
pred = net(samples)
v,i = torch.max(pred, 1)

In [12]:
i

tensor([0, 2, 0, 0, 0, 4, 5, 4, 4, 4, 4, 5, 5, 3, 4, 3])

In [13]:
torch.sum(labels - i)

tensor(0)

In [12]:
hands = mp.solutions.hands.Hands(static_image_mode=False,
                         max_num_hands=2,
                         min_tracking_confidence=0.5,
                         min_detection_confidence=0.5)

mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def img_text(img, text, pos):
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)

In [ ]:
leftkulak.txt
leftladon.txt
l_arm0.txt
rkulak.txt
rladon.txt
r_arm0.txt

In [ ]:
#watch video
names = ['l_fist', 'l_palm', 'l_trash', 'r_fist', 'r_palm', 'r_trash']
cap = cv2.VideoCapture(0)
while(True):
    _, img = cap.read()
    h, w, _ = img.shape
    img = cv2.flip(img, 1)
    # hands
    t = time.time()
    result = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    hand_time = time.time() - t
   
    if result.multi_hand_landmarks:
        for_model = []
        for id, lm in enumerate(result.multi_hand_landmarks[0].landmark):
            cx, cy = int(lm.x*w), int(lm.y*h)
            cv2.circle(img, (cx, cy), 5, (0, 255, 0))
            for_model.append(lm.x)
            for_model.append(lm.y)
        for_model = torch.tensor([for_model]).type(torch.float)
        pred = net(for_model)
        v,i = torch.max(pred, 1)
        img_text(img, str(names[i.item()]), (100, 30))
        if(i.item == 4):
            pyautogui.press('space') 
    cv2.imshow("1",img)
    if cv2.waitKey(1)>-1:
        break
    
cv2.destroyAllWindows()

In [24]:
import pyautogui